<a href="https://colab.research.google.com/github/KT2001/Siamese-Model-for-Omniglot-Dataset/blob/main/Omniglot_Dataset_OneShotProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import necessary libraries
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# import Tensorflow libraries
import tensorflow as tf
import tensorflow.keras.models as models
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten, Lambda
from tensorflow.compat.v1 import ConfigProto
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import time
import uuid
import cv2

import scipy.ndimage as ndi
import sys
import imageio
from imageio import imread

from sklearn.utils import shuffle

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
!nvidia-smi

Mon Dec 12 14:06:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    30W /  70W |    312MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip /content/drive/MyDrive/Siamese/Omniglot/images_background.zip

In [ ]:
!unzip /content/drive/MyDrive/Siamese/Omniglot/images_evaluation.zip

In [ ]:
num_classes_background = {}
sum = 0
dir = os.listdir('/content/images_background')
print(f"Number of classes: {len(dir)} \n")
for filenames in dir:
  num_classes_background[filenames] = len(os.listdir(f"{'/content/images_background'}/{filenames}"))
  sum = num_classes_background[filenames]+sum
  print(f"{filenames}: {(num_classes_background[filenames])}")
print(f"\nTotal number of images: {sum}")

Number of classes: 30 

Alphabet_of_the_Magi: 20
N_Ko: 33
Tagalog: 17
Balinese: 24
Cyrillic: 33
Syriac_(Estrangelo): 23
Mkhedruli_(Georgian): 41
Blackfoot_(Canadian_Aboriginal_Syllabics): 14
Futurama: 26
Bengali: 46
Asomtavruli_(Georgian): 40
Hebrew: 22
Armenian: 41
Tifinagh: 55
Ojibwe_(Canadian_Aboriginal_Syllabics): 14
Inuktitut_(Canadian_Aboriginal_Syllabics): 16
Arcadian: 26
Anglo-Saxon_Futhorc: 29
Japanese_(katakana): 47
Burmese_(Myanmar): 34
Korean: 40
Japanese_(hiragana): 52
Sanskrit: 42
Latin: 26
Malay_(Jawi_-_Arabic): 40
Greek: 24
Grantha: 43
Braille: 26
Gujarati: 48
Early_Aramaic: 22

Total number of images: 964


In [ ]:
num_classes_evaluation = {}
sum = 0
dir = os.listdir('/content/images_evaluation')
print(f"Number of classes: {len(dir)} \n")
for filenames in dir:
  num_classes_evaluation[filenames] = len(os.listdir(f"{'/content/images_evaluation'}/{filenames}"))
  sum = num_classes_evaluation[filenames]+sum
  print(f"{filenames}: {(num_classes_evaluation[filenames])}")
print(f"\nTotal number of images: {sum}")

Number of classes: 20 

Syriac_(Serto): 23
Tengwar: 25
Ge_ez: 26
Malayalam: 47
Sylheti: 28
Kannada: 41
Aurek-Besh: 26
Angelic: 20
Atemayar_Qelisayer: 26
Avesta: 26
Old_Church_Slavonic_(Cyrillic): 45
Tibetan: 42
Mongolian: 30
Glagolitic: 45
Oriya: 46
Gurmukhi: 45
Atlantean: 26
Keble: 26
Manipuri: 40
ULOG: 26

Total number of images: 659


In [ ]:
train_path = os.path.join('/content/images_background')
val_path = os.path.join('/content/images_evaluation')

In [ ]:
def aug(img):
  img = tf.image.resize(img, size=[105, 105])
  img = tf.image.rgb_to_grayscale(img)
  img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
  img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
  # img = tf.image.stateless_random_crop(img, size=(20,20,3), seed=(1,2))
  img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
  #img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
  #img = tf.image.stateless_random_saturation(img, lower=0.9,upper=1, seed=(np.random.randint(100),np.random.randint(100)))
  
  return img


In [ ]:
def load_images_from_dir(path, n=0):
  X = []

  # we load every alphabet and seperately add them to tensors
  for alphabet in os.listdir(path):
    print(f"loading the alphabet path:{alphabet}")
    alpha_path = os.path.join(path, alphabet)

    ## each character is in a seperate folder
    for letter in os.listdir(alpha_path):
      category_image = []
      letter_path = os.path.join(alpha_path, letter)

      if not os.path.isdir(letter_path):
        continue
      
      ### read every image in this directory
      for filename in os.listdir(letter_path):
        image_path = os.path.join(letter_path, filename)
        image = cv2.imread(image_path)
        ### Image preprocessing
        image = image/255
        image = 1-image
        aug_image = aug(image)
        category_image.append(aug_image)

      try:
        X.append(np.stack(category_image))
      # edge case - last one
      except ValueError as e:
        print(f"{e} /nerror: category_image {category_image}")
  
  x = np.stack(X)
  return x

In [ ]:
1# loading the training set
xtrain = load_images_from_dir(train_path)
print(xtrain.shape)

loading the alphabet path:Alphabet_of_the_Magi
loading the alphabet path:N_Ko
loading the alphabet path:Tagalog
loading the alphabet path:Balinese
loading the alphabet path:Cyrillic
loading the alphabet path:Syriac_(Estrangelo)
loading the alphabet path:Mkhedruli_(Georgian)
loading the alphabet path:Blackfoot_(Canadian_Aboriginal_Syllabics)
loading the alphabet path:Futurama
loading the alphabet path:Bengali
loading the alphabet path:Asomtavruli_(Georgian)
loading the alphabet path:Hebrew
loading the alphabet path:Armenian
loading the alphabet path:Tifinagh
loading the alphabet path:Ojibwe_(Canadian_Aboriginal_Syllabics)
loading the alphabet path:Inuktitut_(Canadian_Aboriginal_Syllabics)
loading the alphabet path:Arcadian
loading the alphabet path:Anglo-Saxon_Futhorc
loading the alphabet path:Japanese_(katakana)
loading the alphabet path:Burmese_(Myanmar)
loading the alphabet path:Korean
loading the alphabet path:Japanese_(hiragana)
loading the alphabet path:Sanskrit
loading the alphab

In [ ]:
# loading the validation set
xval = load_images_from_dir(val_path)
print(xval.shape)

loading the alphabet path:Syriac_(Serto)
loading the alphabet path:Tengwar
loading the alphabet path:Ge_ez
loading the alphabet path:Malayalam
loading the alphabet path:Sylheti
loading the alphabet path:Kannada
loading the alphabet path:Aurek-Besh
loading the alphabet path:Angelic
loading the alphabet path:Atemayar_Qelisayer
loading the alphabet path:Avesta
loading the alphabet path:Old_Church_Slavonic_(Cyrillic)
loading the alphabet path:Tibetan
loading the alphabet path:Mongolian
loading the alphabet path:Glagolitic
loading the alphabet path:Oriya
loading the alphabet path:Gurmukhi
loading the alphabet path:Atlantean
loading the alphabet path:Keble
loading the alphabet path:Manipuri
loading the alphabet path:ULOG
(659, 20, 105, 105, 1)


In [ ]:
def get_batches(batch_size, data=xtrain):
  #tf.config.run_functions_eagerly(True)
  data = np.squeeze(data)
  n_classes, n_examples, w, h = data.shape

  # initialize the data we return in memory
  pairs = [np.zeros((batch_size, h, w, 1)) for i in range(2)]

  # make the target vector with half same half other category
  targets = np.zeros((batch_size,))
  targets[batch_size//2:] = 1

  # pick the categories of characters we will return 
  categories = np.random.choice(n_classes, size=(batch_size,), replace = False)

  for i in range(batch_size):
    category = categories[i]

    ## choose two indices from the amount of examples 
    id1 = np.random.randint(0, n_examples)
    id2 = np.random.randint(0, n_examples)

    if targets[i] == 0:
      category_2 = category # if target is set pick from same class 
    else:
      # pick new classes by picking random number 
      category_2 = (category + np.random.randint(1, n_classes)) % n_classes

    pairs[0][i,:,:,:] = data[category, id1].reshape(w, h, 1)
    pairs[1][i,:,:,:] = data[category_2, id2].reshape(w, h, 1)

  return pairs, targets

In [ ]:
from sklearn.utils import shuffle

In [ ]:
data = xval
data = np.squeeze(data)
data.shape

(659, 20, 105, 105)

In [ ]:
def make_oneshot_task(N, data=xval):
  n_val, n_ex_val, w, h = data.shape

  categories = np.random.choice(n_val, size=(N,), replace=False)
  true_category = categories[0]

  indices = np.random.randint(0, n_ex_val, size = (N,))

  ex1, ex2 = np.random.choice(n_ex_val, replace=False, size = (2,))

  # create a list with same image N times as test image
  test_image = np.asarray([data[true_category, ex1, :, :]]*N).reshape(N, w, h, 1)
  support_set = data[categories, indices,:,:]
  support_set[0,:,:] = data[true_category, ex2]
  support_set = support_set.reshape(N, w, h, 1)
  targets = np.zeros((N,))
  targets[0] = 1
  targets, test_image, support_set = shuffle(targets, test_image, support_set)
  pairs = [test_image,support_set]

  return pairs, targets

In [ ]:
def generate(batch_size, s="train"):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batches(batch_size, s)
        yield (pairs, targets)

In [ ]:
def test_oneshot(model,N,k=1,verbose=0, data=xval):
    #Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks
    data = np.squeeze(data)
    n_correct = 0
    for i in range(k):
      inputs, targets = make_oneshot_task(N, data)
      probs = model.predict(inputs)
      if np.argmax(probs) == 0:
          n_correct+=1
    percent_correct = (100.0*n_correct / k)
    return percent_correct

In [ ]:
def W_init(shape, dtype=None):
    """Initialize weights as in paper"""
    values = np.random.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values, dtype=dtype)
#//TODO: figure out how to initialize layer biases in keras.
def b_init(shape, dtype=None):
    """Initialize bias as in paper"""
    values=np.random.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values, dtype=dtype)

In [ ]:
input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convnet to use in each siamese 'leg'
convnet = models.Sequential()
convnet.add(Conv2D(64,(10,10),activation='relu',input_shape=input_shape, kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(7,7),activation='relu',
                   kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(256,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))

#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#layer to merge two encoded inputs with the l1 distance between them
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(L1_distance)
siamese_net = models.Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(0.00006)

In [ ]:
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()

38951745

In [ ]:
siamese_net.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 105, 105, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 105, 105, 1  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 4096)         38947648    ['input_1[0][0]',                
                                                                  'input_2[0][0]']            

In [ ]:
# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 20000 # No. of training iterations
N_way = 20 # how many classes for testing one-shot tasks
n_val = 250 # how many one-shot tasks to validate on
best = -1

In [ ]:
model_path = '/content/model_path'

In [ ]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    inputs,targets = get_batches(batch_size)
    loss = siamese_net.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(siamese_net, N_way, n_val, verbose=True)
        siamese_net.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 29ms/step

 ------------- 

Time for 16400 iterations: 55.64723720550537 mins
Train Loss: 0.20985278487205505
1/1 [==============================] - 0s 25ms/step

 ------------- 

Time for 16600 iterations: 56.32914522488912 mins
Train Loss: 0.2104818969964981
1/1 [==============================] - 0s 20ms/step

 ------------- 

Time for 16800 iterations: 57.00834828217824 mins
Train Loss: 0.1517343819141388
1/1 [==============================] - 0s 20ms/step

 ------------- 

Time for 17000 iterations: 57.7145731250445 mins
Train Loss: 0.3150440454483032
1/1 [==============================] - 0s 25ms/step

 ------------- 

Time for 17200 iterations: 58.39331443309784 mins
Train Loss: 0.15473473072052002
1/1 [==============================] - 0s 21ms/step

 ------------- 

Time for 17400 iterations: 59.07859421571096 mins
Train Loss: 0.22769173979759216
1/1 [==============================] - 0s

In [ ]:
best

9.6